# Init

In [ ]:
import numpy as np
import logging
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%cd ..
%load_ext autoreload

import plotly.graph_objects as go

In [ ]:
%autoreload 2
from src.MapData import MapData
from src.LineFinder import LineFinder

from src.plotting_functions import plot_embedding, plot_single_variable_map

In [ ]:
map_data = MapData()
map_data.get_metadata()
map_data.load_wavelenths()
map_data.load_all_data()
map_data.trim_spectra(64)

In [ ]:
map_data.spectra = map_data.spectra[:10000,:].copy()

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def rolling_min(arr, window_width):
    window = sliding_window_view(
        arr, 
        (window_width,),
        axis=len(arr.shape) - 1
    )
    return np.amin(window, axis=len(arr.shape))

In [ ]:
# %%timeit -r 3
min_window_size = 50
smooth_window_size = 100

local_minima = rolling_min(
    arr=np.hstack(
        [map_data.spectra[:,0][:,np.newaxis]] * ((min_window_size + smooth_window_size) // 2)\
            + [map_data.spectra]
            + [map_data.spectra[:,-1][:,np.newaxis]] * ((min_window_size + smooth_window_size) // 2)
    ),
    window_width=min_window_size
)

In [ ]:
# %%timeit -r 3
min_window_size = 50
smooth_window_size = 100

kernel = np.arange(-smooth_window_size//2,smooth_window_size//2 + 1,1)
sigma = smooth_window_size // 4
kernel = np.exp(-(kernel ** 2) / (2 * sigma**2))
kernel /= kernel.sum()

# local_minima = np.apply_along_axis(
#     arr=np.hstack(
#         [map_data.spectra[:,0][:,np.newaxis]] * ((min_window_size + smooth_window_size) // 2)\
#             + [map_data.spectra]
#             + [map_data.spectra[:,-1][:,np.newaxis]] * ((min_window_size + smooth_window_size) // 2)
#     ),
#     func1d=rolling_min,
#     axis=1,
#     window_width=min_window_size
# )

smoothed_minima = np.apply_along_axis(
    arr=local_minima,
    func1d=np.convolve,
    axis=1,
    v=kernel,
    mode='valid'
)

In [ ]:
smoothed_minima = np.convolve(
    a=local_minima,
    # axis=1,
    v=kernel,
    mode='valid'
)

In [ ]:
local_minima.shape

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=map_data.wvl,
        y=map_data.spectra[0,:]
    )
)
fig.add_trace(
    go.Scatter(
        x=map_data.wvl,
        y=np.squeeze(local_minima[0,min_window_size:]),
        name='minima'
    )
)
fig.add_trace(
    go.Scatter(
        x=map_data.wvl,
        y=smoothed_minima[0,:],
        name='baseline'
    )
)


In [ ]:
smoothed_minima = smoothed_minima[
    :,
    :-(smoothed_minima.shape[1] - map_data.spectra.shape[1])
]

In [ ]:
print(map_data.spectra.shape)
print(smoothed_minima.shape)

In [ ]:
map_data.spectra = np.subtract(
    map_data.spectra,
    smoothed_minima
)